In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pandas as pd
import scipy.interpolate as interp

def LoadResults(config):
    
    rad, angle, sample, slice_ = np.meshgrid(
        config['rad'], 
        config['angle'],
        config['sample'],
        config['slice']
    )
    
    result_list = []
    for i, group in enumerate(zip(rad.flatten(), angle.flatten(), sample.flatten(), slice_.flatten())):
        name = f'{config["date_result"]}_angle{group[1]}_rad{group[0]}_nsample{group[2]}_nslice{group[3]}.npy'
        result_array = np.load(os.path.join(config['results'], name))
        
        result_entry = {
            'grid-size': result_array[0],
            'match-ratio': result_array[1],
            'rad': group[0],
            'angle': group[1],
            'sample': group[2],
            'slice': group[3],
        }
        result_list.append(result_entry)
        
    return pd.DataFrame.from_records(result_list)

def SmoothRawMatchRatio(grid_size, match):
    
    unique_grid_size = np.flip(np.unique(grid_size))

    unique_match = np.zeros(unique_grid_size.size)

    for i, size in enumerate(unique_grid_size):
        unique_match[i] = match[np.argwhere(grid_size == size).squeeze()].max()


    select_grid_size = []
    select_match = []

    last_match = -1
    for i, imatch in enumerate(np.flip(unique_match)):
        if imatch >= last_match:
            select_grid_size.append(np.flip(unique_grid_size)[i])
            select_match.append(imatch)

            last_match = imatch

    select_grid_size = np.array(select_grid_size)
    select_match = np.array(select_match)

    return (select_grid_size, select_match)
    

PATH = '/storage/home/adz6/group/project'
RESULTS = os.path.join(PATH, 'results/matched_filter/dense_grid/match_ratios')
PLOTS = os.path.join(PATH, 'plots/analysis/matched_filter/dense_grid/match_ratios')

sns.set_theme(context='talk')

In [ ]:
config = {
    'rad': [1, 2, 3, 4],
    'angle': [86.0, 87.0, 88.0],
    'sample': [4096],
    'slice': [2],
    'date_result': 20220127,
    'date_ran': 20220128,
    'results': RESULTS,
    'plots': PLOTS,
}

In [ ]:
results = LoadResults(config)

In [ ]:
results

# plot raw match ratios for same angles, different radii

In [ ]:
angle = 88.0
sample = 4096
slice_ = 2

fig = plt.figure(figsize=(13, 8))

ax = fig.add_subplot(1,1,1)


for i, rad in enumerate(results[results['angle'] == angle]['rad'].unique()):
    
    x = results[(results['angle'] == angle) & (results['rad'] == rad)]['grid-size'].iloc[0]
    y = results[(results['angle'] == angle) & (results['rad'] == rad)]['match-ratio'].iloc[0]
    
    ax.plot(x, y, '.', label=f'R = {rad} cm')
    
ax.set_xscale('log')
ax.set_xlabel('Grid Size')
ax.set_ylabel('Match Ratio')

ax.legend(loc=4)
ax.set_title(f'Mean Match Ratios, Pitch: {angle} deg, Samples: {slice_ * sample}')

plt.tight_layout()
name = f'{config["date_ran"]}_angle{angle}_sample{slice_ * sample}_rad_sweep.png'
plt.savefig(os.path.join(PLOTS, name))

# plot raw match ratios for diff angles, same radii

In [ ]:
rad = 4
sample = 4096
slice_ = 2

fig = plt.figure(figsize=(13, 8))

ax = fig.add_subplot(1,1,1)


for i, angle in enumerate(results[results['rad'] == rad]['angle'].unique()):
    
    x = results[(results['angle'] == angle) & (results['rad'] == rad)]['grid-size'].iloc[0]
    y = results[(results['angle'] == angle) & (results['rad'] == rad)]['match-ratio'].iloc[0]
    
    ax.plot(x, y, '.', label=f'Pitch = {angle} deg')
    
ax.set_xscale('log')
ax.set_xlabel('Grid Size')
ax.set_ylabel('Match Ratio')

ax.legend(loc=4)
ax.set_title(f'Mean Match Ratios, Rad: {rad} cm, Samples: {slice_ * sample}')

plt.tight_layout()
name = f'{config["date_ran"]}_rad{rad}_sample{slice_ * sample}_pitch_sweep.png'
plt.savefig(os.path.join(PLOTS, name))

# smooth out the raw mean match ratios, plot for diff angles, same radii

In [ ]:
rad = 3
sample = 4096
slice_ = 2



colors = sns.color_palette('deep')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
for i, angle in enumerate(results[results['rad'] == rad]['angle'].unique()):
    
    x = results[(results['angle'] == angle) & (results['rad'] == rad)]['grid-size'].iloc[0]
    y = results[(results['angle'] == angle) & (results['rad'] == rad)]['match-ratio'].iloc[0]
    
    smooth_match = SmoothRawMatchRatio(x, y)
    
    x_smooth = smooth_match[0]
    y_smooth = smooth_match[1]
    
    #ax.plot(x, y, '.', label=f'Pitch = {angle} deg', color=colors[i])
    ax.plot(x_smooth, y_smooth, label=f'Pitch = {angle} deg', color=colors[i])
    
ax.set_xscale('log')
ax.set_xlabel('Grid Size')
ax.set_ylabel('Match Ratio')

ax.legend(loc=4)
ax.set_title(f'Smooth Match Ratios, Rad: {rad} cm, Samples: {slice_ * sample}')

plt.tight_layout()
name = f'{config["date_ran"]}_rad{rad}_sample{slice_ * sample}_pitch_sweep_smooth.png'
plt.savefig(os.path.join(PLOTS, name))

# smooth out the raw mean match ratios, plot for same angles, diff radii

In [ ]:
angle = 88.0
sample = 4096
slice_ = 2

colors = sns.color_palette('deep')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

for i, rad in enumerate(results[results['angle'] == angle]['rad'].unique()):
    
    x = results[(results['angle'] == angle) & (results['rad'] == rad)]['grid-size'].iloc[0]
    y = results[(results['angle'] == angle) & (results['rad'] == rad)]['match-ratio'].iloc[0]
    
    smooth_match = SmoothRawMatchRatio(x, y)
    
    x_smooth = smooth_match[0]
    y_smooth = smooth_match[1]
    
    #ax.plot(x, y, '.', label=f'Pitch = {angle} deg', color=colors[i])
    ax.plot(x_smooth, y_smooth, label=f'R = {rad} cm', color=colors[i])
    
ax.set_xscale('log')
ax.set_xlabel('Grid Size')
ax.set_ylabel('Match Ratio')

ax.legend(loc=4)
ax.set_title(f'Smooth Match Ratios, Pitch: {angle} deg, Samples: {slice_ * sample}')

plt.tight_layout()
name = f'{config["date_ran"]}_pitch{angle}_sample{slice_ * sample}_rad_sweep_smooth.png'
plt.savefig(os.path.join(PLOTS, name))